In [2]:
import pandas as pd
from collections import Counter
import numpy as np

In [3]:
base_df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",
                        usecols=['SessionID', 'TIMESTAMP', 'URL_FILE', ])
base_df.rename(columns={'URL_FILE':'Activity'}, inplace=True)
base_df["TIMESTAMP"] = pd.to_datetime(base_df["TIMESTAMP"], infer_datetime_format=True)
base_df = base_df.sort_values(["SessionID", "TIMESTAMP"])

In [5]:
# unique sessions
uni = base_df["SessionID"].unique()
len(uni)

660270

We want to remove a activity is the activity before it is the same

In [7]:
def create_clear_sequence(_df:pd.DataFrame):
  df = _df.copy()
  # group by session and create a column with the next activity
  df["Act_before"] = df.groupby("SessionID")["Activity"].shift(periods=+1)
  df['same_act'] = df['Activity'] == df['Act_before']
  # remove row if the activity is the same as the previous one
  df = df[df['same_act'] == False]
  # drop the columns that are not needed anymore
  df = df.drop(columns=['same_act', 'Act_before'])
  return df
  

In [8]:
no_repeated_df = create_clear_sequence(base_df)
no_repeated_df

,SessionID,TIMESTAMP,Activity
3273278,46,2015-11-06 08:07:22.780,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken
5333642,46,2015-11-06 08:07:40.767,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...
3733243,46,2015-11-06 08:07:51.390,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...
5904405,46,2015-11-06 08:08:06.003,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...
1071622,92,2015-10-14 14:59:37.753,/werk_nl/werknemer/mijn_werkmap
...,...,...,...
1400504,55314390,2016-02-28 09:26:57.507,/werk_nl/werknemer/home
2213018,55314605,2016-02-27 10:13:43.503,/xdocs/ux-frontend/fonts/uwvsanlig-webfont.woff
3084818,55314605,2016-02-27 10:13:51.350,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...
1613546,55314751,2016-02-28 08:17:15.947,/werk_nl/werknemer/werkmap


In [ ]:
# export the dataframe to a csv file
no_repeated_df.to_csv("no_repeated.csv", index=False)